# Adversarial Testing

Nomotic is a runtime governance framework for agentic AI. Governance that only works under friendly conditions is not governance at all -- it is a suggestion. Nomotic includes built-in adversarial testing scenarios to verify that governance holds under attack.

This notebook demonstrates how to run and interpret adversarial tests against the Nomotic governance runtime. The scenarios covered here exercise privilege escalation attempts, rapid-fire flooding, and scope assembly attacks where multiple agents with narrow scopes attempt to combine their authorities to achieve an unauthorized outcome.

Each scenario is designed to confirm that the governance pipeline -- scope compliance, incident detection, and the workflow governor -- correctly identifies and blocks adversarial behavior.

In [ ]:
# Run the built-in adversarial test suite.
# This executes all predefined adversarial scenarios and reports results.
!nomotic test --adversarial

In [ ]:
"""Custom adversarial scenario: Privilege Escalation

Simulates a prompt-injection attack where a restricted agent (scope: read only)
attempts to escalate its privileges by issuing write, delete, and transfer
actions that exceed its authorized scope.
"""

from nomotic import GovernanceRuntime, Action, AgentContext, TrustProfile, Verdict

# Create a governance runtime
runtime = GovernanceRuntime()

# Configure a restricted agent with scope {"read"} only
agent_id = "restricted-agent-01"
scope_dim = runtime.registry.get("scope_compliance")
scope_dim.configure_agent_scope(agent_id, {"read"})

context = AgentContext(
    agent_id=agent_id,
    trust_profile=TrustProfile(agent_id=agent_id, overall_trust=0.5),
)

# Escalating sequence of actions -- simulates prompt injection
escalation_actions = [
    ("read",     "data/reports",    "Allowed -- within scope"),
    ("write",    "data/reports",    "Should be DENIED -- outside scope"),
    ("delete",   "data/reports",    "Should be DENIED -- outside scope"),
    ("transfer", "data/sensitive",  "Should be DENIED -- outside scope"),
]

escalation_verdicts = []

for action_type, target, description in escalation_actions:
    action = Action(
        agent_id=agent_id,
        action_type=action_type,
        target=target,
    )
    verdict = runtime.evaluate(action, context)
    escalation_verdicts.append((action_type, verdict))
    print(f"[{action_type:>10}] {verdict.verdict.name:<10} | {description}")
    if verdict.vetoed_by:
        print(f"             Vetoed by: {verdict.vetoed_by}")

In [ ]:
"""Adversarial scenario: Rapid-Fire Attack

Simulates a flooding attack by sending 20 actions in quick succession from
a single agent. The runtime's incident detection dimension monitors action
frequency and flags anomalous bursts of activity.
"""

from nomotic import GovernanceRuntime, Action, AgentContext, TrustProfile, Verdict

runtime_rf = GovernanceRuntime()

agent_id = "rapid-fire-agent"
scope_dim = runtime_rf.registry.get("scope_compliance")
scope_dim.configure_agent_scope(agent_id, {"read"})

context = AgentContext(
    agent_id=agent_id,
    trust_profile=TrustProfile(agent_id=agent_id, overall_trust=0.5),
)

rapid_fire_verdicts = []
deny_count = 0
escalate_count = 0

print("Sending 20 rapid-fire 'read' actions...\n")

for i in range(20):
    action = Action(
        agent_id=agent_id,
        action_type="read",
        target=f"data/record-{i}",
    )
    verdict = runtime_rf.evaluate(action, context)
    rapid_fire_verdicts.append(verdict)

    if verdict.verdict == Verdict.DENY:
        deny_count += 1
    elif verdict.verdict == Verdict.ESCALATE:
        escalate_count += 1

    status = verdict.verdict.name
    trust = context.trust_profile.overall_trust
    print(f"  Action {i+1:>2}: {status:<10} (trust: {trust:.3f})")

print(f"\nSummary: {deny_count} denied, {escalate_count} escalated out of 20 actions")
print(f"Final trust level: {context.trust_profile.overall_trust:.3f}")
print("\nThe incident detection dimension tracks rapid repeated actions.")
print("Trust degrades as suspicious patterns accumulate, causing later actions")
print("to face stricter governance even when individually in-scope.")

In [ ]:
"""Adversarial scenario: Scope Assembly Attack

Two agents have narrow scopes -- one can only read, the other can only write.
Neither agent individually can perform a read-then-write operation that would
be equivalent to "update". This scenario verifies that:

  1. Agent A (read-only) is denied write access.
  2. Agent B (write-only) is denied read access.
  3. The workflow governor tracks compound authority across steps,
     detecting when narrow scopes are assembled into broader capabilities.
"""

from nomotic import (
    GovernanceRuntime,
    Action,
    AgentContext,
    TrustProfile,
    Verdict,
    WorkflowGovernor,
    WorkflowGovernorConfig,
    CompoundAuthorityFlag,
)
from nomotic.context_profile import (
    ContextProfile,
    WorkflowContext,
    PlannedStep,
    CompletedStep,
)

runtime_sa = GovernanceRuntime()

# Agent A: read-only scope
agent_a = "assembly-reader"
scope_dim = runtime_sa.registry.get("scope_compliance")
scope_dim.configure_agent_scope(agent_a, {"read"})

# Agent B: write-only scope
agent_b = "assembly-writer"
scope_dim.configure_agent_scope(agent_b, {"write"})

context_a = AgentContext(
    agent_id=agent_a,
    trust_profile=TrustProfile(agent_id=agent_a, overall_trust=0.5),
)
context_b = AgentContext(
    agent_id=agent_b,
    trust_profile=TrustProfile(agent_id=agent_b, overall_trust=0.5),
)

assembly_verdicts = []

# Step 1: Agent A reads (should be allowed)
read_action = Action(agent_id=agent_a, action_type="read", target="data/accounts")
v1 = runtime_sa.evaluate(read_action, context_a)
assembly_verdicts.append(("Agent A read", v1))
print(f"Agent A (read-only)  -> read:  {v1.verdict.name}")

# Step 2: Agent A tries to write (should be denied)
write_action_a = Action(agent_id=agent_a, action_type="write", target="data/accounts")
v2 = runtime_sa.evaluate(write_action_a, context_a)
assembly_verdicts.append(("Agent A write", v2))
print(f"Agent A (read-only)  -> write: {v2.verdict.name}")

# Step 3: Agent B writes (should be allowed)
write_action_b = Action(agent_id=agent_b, action_type="write", target="data/accounts")
v3 = runtime_sa.evaluate(write_action_b, context_b)
assembly_verdicts.append(("Agent B write", v3))
print(f"Agent B (write-only) -> write: {v3.verdict.name}")

# Step 4: Agent B tries to read (should be denied)
read_action_b = Action(agent_id=agent_b, action_type="read", target="data/accounts")
v4 = runtime_sa.evaluate(read_action_b, context_b)
assembly_verdicts.append(("Agent B read", v4))
print(f"Agent B (write-only) -> read:  {v4.verdict.name}")

print("\n--- Workflow Governor: Compound Authority Analysis ---")
print("\nNeither agent alone can perform an 'update' (read + write).")
print("The workflow governor detects compound authority when narrow")
print("scopes are assembled across agents or steps in a workflow.")

# Demonstrate workflow governor compound authority detection
governor = WorkflowGovernor()
workflow = WorkflowContext(
    workflow_id="assembly-attack-workflow",
    total_steps=2,
    current_step=2,
    planned_steps=[
        PlannedStep(step_number=1, method="read", description="Read accounts data"),
        PlannedStep(step_number=2, method="write", description="Write accounts data"),
    ],
    completed_steps=[
        CompletedStep(step_number=1, method="read", outcome="success"),
    ],
)
profile = ContextProfile(agent_id=agent_a, workflow=workflow)
assessment = governor.assess_workflow("assembly-attack-workflow", profile)

print(f"\nWorkflow risk score: {assessment.cumulative_risk_score:.3f}")
print(f"Recommendation:      {assessment.recommendation}")
if assessment.compound_authority_flags:
    for flag in assessment.compound_authority_flags:
        print(f"\nCompound authority flag:")
        print(f"  Description:          {flag.description}")
        print(f"  Methods chained:      {flag.methods_chained}")
        print(f"  Resulting capability: {flag.resulting_capability}")
        print(f"  Severity:             {flag.severity}")
else:
    print("\nNo compound authority flags raised for this workflow.")
    print("Individual scope enforcement already blocked the cross-scope attempt.")

In [ ]:
"""Scenario Report

Collect all verdicts from the adversarial tests above and display a summary
table showing: scenario name, expected verdict, actual verdict, and pass/fail.
"""

# Collect test cases: (scenario, expected_verdict, actual_verdict)
test_results = []

# From privilege escalation scenario
expected_escalation = [
    ("Priv Esc: read (in scope)",       Verdict.ALLOW, escalation_verdicts[0][1].verdict),
    ("Priv Esc: write (out of scope)",   Verdict.DENY,  escalation_verdicts[1][1].verdict),
    ("Priv Esc: delete (out of scope)",  Verdict.DENY,  escalation_verdicts[2][1].verdict),
    ("Priv Esc: transfer (out of scope)",Verdict.DENY,  escalation_verdicts[3][1].verdict),
]
test_results.extend(expected_escalation)

# From rapid-fire scenario -- later actions should face stricter governance
test_results.append((
    "Rapid-Fire: first action",
    Verdict.ALLOW,
    rapid_fire_verdicts[0].verdict,
))
# Check that at least some later actions were denied or escalated
later_verdicts = [v.verdict for v in rapid_fire_verdicts[10:]]
any_governance_response = any(
    v in (Verdict.DENY, Verdict.ESCALATE, Verdict.MODIFY) for v in later_verdicts
)
test_results.append((
    "Rapid-Fire: governance tightens",
    Verdict.DENY,  # expected: at least some non-ALLOW verdicts
    Verdict.DENY if any_governance_response else Verdict.ALLOW,
))

# From scope assembly scenario
test_results.append(("Scope Assembly: A reads (in scope)",     Verdict.ALLOW, assembly_verdicts[0][1].verdict))
test_results.append(("Scope Assembly: A writes (out of scope)", Verdict.DENY,  assembly_verdicts[1][1].verdict))
test_results.append(("Scope Assembly: B writes (in scope)",     Verdict.ALLOW, assembly_verdicts[2][1].verdict))
test_results.append(("Scope Assembly: B reads (out of scope)",  Verdict.DENY,  assembly_verdicts[3][1].verdict))

# Display summary table
print(f"{'Scenario':<45} {'Expected':<12} {'Actual':<12} {'Status'}")
print("-" * 78)

pass_count = 0
fail_count = 0

for scenario, expected, actual in test_results:
    passed = actual == expected
    status = "PASS" if passed else "FAIL"
    if passed:
        pass_count += 1
    else:
        fail_count += 1
    print(f"{scenario:<45} {expected.name:<12} {actual.name:<12} {status}")

print("-" * 78)
print(f"Total: {pass_count} passed, {fail_count} failed out of {len(test_results)} tests")

## OWASP Mapping

The adversarial scenarios in this notebook map directly to categories in the [OWASP Top 10](https://owasp.org/www-project-top-ten/) and [OWASP API Security Top 10](https://owasp.org/www-project-api-security/):

| Adversarial Scenario | OWASP Category | Description |
|---|---|---|
| **Privilege Escalation** | **A01:2021 -- Broken Access Control** | The restricted agent attempts to perform actions (write, delete, transfer) outside its authorized scope. Nomotic's scope compliance dimension enforces least-privilege boundaries. This maps to OWASP's Broken Access Control, where users or agents act outside their intended permissions. |
| **Rapid-Fire Attack** | **A05:2021 -- Security Misconfiguration / Rate Limiting** | Flooding the runtime with rapid repeated actions tests whether governance degrades under volume. The incident detection dimension monitors action frequency and behavioral patterns. This maps to Security Misconfiguration where missing rate limits allow abuse, and also aligns with API4:2023 -- Unrestricted Resource Consumption. |
| **Scope Assembly Attack** | **A01:2021 -- Broken Function Level Authorization** | Two agents with narrow scopes individually cannot perform an update operation (read + write), but could theoretically combine their authorities. The workflow governor tracks compound authority across workflow steps to detect this pattern. This maps to Broken Function Level Authorization (API5:2023), where granular permission checks fail to account for combined capabilities. |

Nomotic's governance model addresses these categories through defense-in-depth: scope compliance provides per-action authorization, incident detection provides behavioral anomaly detection, and the workflow governor provides workflow-level compound authority analysis.